# Your name: 

This notebook implements timeloop/accelergy-based energy estimation for the neural network model you trained. This part has to be run with the docker we provide, and does not require GPU support. 

One strategy to reduce the profiling time is to design a model with repeated layers since layers with the same architecture only need one time of profiling.
The profiler will also automatically save the information of profiled layers to a .json file specifiled by `profiled_lib_dir`, so that next time the same layer is profiled, the results can be obtained immediately. 


### 1. Load the model

In [125]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Change this model class to the architecture you used
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#net = Net()

In [126]:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
 #       self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

class ResNetEff(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNetEff, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.layer1 = self._make_layer(block, self.in_planes, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 16, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 16, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 16, num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, 32, num_blocks[4], stride=2)
        self.layer6 = self._make_layer(block, 64, num_blocks[5], stride=2)
        self.layer7 = self._make_layer(block, 128, num_blocks[6], stride=2)
        self.linear = nn.Linear(128*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        
        out = self.layer6(out)
        #print(out.shape)
        out = self.layer7(out)
        #print(out.shape)
        out = F.avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out    

def ResNetMake():
    return ResNetEff(BasicBlock, [2, 2, 2, 2, 2, 2, 2])    
    
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())
    
    
net = ResNetMake()

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam([var1, var2], lr=0.0001)

### 2. Run the Profiler to estimate the peak activation size

In [127]:
from profiler import count_activation_size
peak_activation_size = count_activation_size(
    net=net,
    input_size=(1, 3, 32, 32),
)

print(f"Peak Activation Sizes: {peak_activation_size} Byte")

Peak Activation Sizes: 131072.0 Byte


### 3. Run the Profiler for Timeloop/Accelergy

In [128]:
from profiler import Profiler
from datetime import date

today = date.today()
sub_dir = "network-" + today.strftime("%b-%d-%Y")

profiler = Profiler(
    top_dir='workloads',
    sub_dir=sub_dir,
    timeloop_dir='simple_weight_stationary',
    model=net,
    input_size=(3, 32, 32),
    batch_size=1,
    convert_fc=True,
    exception_module_names=[],
    profiled_lib_dir=f"profiled_lib.json"
)

layer_wise, overall = profiler.profile()

for layer_id, info in layer_wise.items():
    print(f"Name: {info['name']} \t Energy: {info['energy']:.2f} \t Cycle: {info['cycle']} \t Number of same architecture layers: {info['num']}")
    
print(f"\nTotal Energy: {overall['total_energy']/1e9:.8f} mj \nTotal Cycles: {overall['total_cycle']/1e6:.8f} Million")

print(f"MACs: {overall['macs']}\nNum of Parameters: {overall['num_params']} \nPeak Activation Size: {overall['activation_size']} Byte")




converting nn.Conv2d and nn.Linear in network-Feb-28-2022 model ...
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer1.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer2.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer3.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer4.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer5.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer6.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer7.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer8.yaml
workload file --> /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer9.yaml
workload file --> /home

100%|██████████| 21/21 [01:21<00:00,  3.89s/it]

timeloop running finished!
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer1 	 Energy: 40243493.46 	 Cycle: 9216 	 Number of same architecture layers: 1
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer2 	 Energy: 60141767.78 	 Cycle: 9216 	 Number of same architecture layers: 4
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer6 	 Energy: 17142877.61 	 Cycle: 2304 	 Number of same architecture layers: 1
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer7 	 Energy: 15346000.33 	 Cycle: 2304 	 Number of same architecture layers: 3
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer8 	 Energy: 1546086.51 	 Cycle: 256 	 Number of same architecture layers: 1
Name: /home/workspace/lab2/workloads/network-Feb-28-2022/network-Feb-28-2022_layer11 	 Energy: 2700128.12 	 Cycle: 576 	 Number of same architecture layers: 1
Name: /home/work